In [1]:
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import copy

from tslib.src import tsUtils
from tslib.src.synthcontrol.syntheticControl import RobustSyntheticControl
from tslib.src.synthcontrol.multisyntheticControl import MultiRobustSyntheticControl

import random
from numpy.linalg import svd, matrix_rank, norm
from sklearn import linear_model
import pickle

In [3]:
def hsvt(X, rank): 
    """
    Input:
        X: matrix of interest
        rank: rank of output matrix
    Output:
        thresholded matrix
    """
    u, s, v = np.linalg.svd(X, full_matrices=False)
    s[rank:].fill(0)
    return np.dot(u*s, v)

def hsvt_df(X, rank): 
    """
    Input:
        X: matrix of interest
        rank: rank of output matrix
    Output:
        thresholded matrix
    """
    u, s, v = np.linalg.svd(X, full_matrices=False)
    s[rank:].fill(0)
    vals = (np.dot(u*s, v))
    return pd.DataFrame(vals, index = X.index, columns = X.columns)

def matrixFromSVD(sk, Uk, Vk, probability=1.0):
    # Jehangir's
    return (1.0/probability) * np.dot(Uk, np.dot(np.diag(sk), Vk.T))

def hsvt_new(X, rank):
    Uk, sk, Vk = np.linalg.svd(X, full_matrices=False)
    Vk = Vk.T
    sk[rank:].fill(0)
    output = matrixFromSVD(sk, Uk, Vk, probability=1.0)
    return output

def hsvt_new_df(df, rank):
    Uk, sk, Vk = np.linalg.svd(df.values, full_matrices=False)
    Vk = Vk.T
    sk[rank:].fill(0)
    output = matrixFromSVD(sk, Uk, Vk, probability=1.0)
    return pd.DataFrame(output, index=df.index, columns=df.columns)

def get_preint_data(X, T0, T, K):
    """
    Input:
        X: N x KT matrix
        T0: pre-int period
        T: total period
        K: number of metrics
    
    Output:
        X_pre: N x KT0 matrix
    """
    X_pre = np.array([])
    for k in range(K): 
        if X.ndim > 1:
            X_temp = X[:, k*T:k*T + T0]
        else:
            X_temp = X[k*T:k*T + T0]
        X_pre = np.hstack([X_pre, X_temp]) if X_pre.size else X_temp
    return X_pre

def get_postint_data(X, T0, T, K):
    """
    Input:
        X: N x KT matrix
        T0: pre-int period
        T: total period
        K: number of metrics
    
    Output:
        X_post: N x K(T-T0) matrix
    """
    X_post = np.array([])
    for k in range(K): 
        if X.ndim > 1:
            X_temp = X[:, k*T+T0:(k+1)*T]
        else:
            X_temp = X[k*T+T0:(k+1)*T]
        X_post = np.hstack([X_post, X_temp]) if X_post.size else X_temp
    return X_post


def pre_post_split(y, T0, T, num_metrics):
        y_pre = get_preint_data(y, T0, T, num_metrics)
        y_post = get_postint_data(y, T0, T, num_metrics)
        return y_pre, y_post


def approximate_rank(X, t=99):
    """
    Input:
        X: matrix of interest
        t: an energy threshold. Default (99%)
        
    Output:
        r: approximate rank of Z
    """
    u, s, v = np.linalg.svd(X, full_matrices=False)
    total_energy = (100*(s**2).cumsum()/(s**2).sum())
    r = list((total_energy>t)).index(True)+1
    return r

def relative_spectrum(X):
    """
    Input:
        X: matrix of interest
        
    Output:
        list: with % of spectrum explained by first eigenvalues of Z
    """
    u, s, v = np.linalg.svd(X, full_matrices=False)
    return (s**2)/((s**2).sum())

def donor_prep(X, t):
    """
    Input:
        X: matrix of interest
        t: threshold
    
    Output:
        thresholded matrix
    """
    r = approximate_rank(X, thresh)
    print("{} SV = {}% of energy".format(r, t))
    X_hsvt = hsvt(X, r)
    return np.abs(X_hsvt.round())

def mse(y, y_pred):
    return np.sum((y - y_pred) ** 2) / len(y)

def rmse(y, y_pred):
    return np.sqrt(mse(y,y_pred))

def mape(y, y_pred):
    mask = (y != 0)
    return np.mean(np.abs((y - y_pred)[mask] / y[mask]))

class mRSC:
    def __init__(self, donor, target, metrics, donor_ids, target_ids, T_0s, singvals): 
        """
        donor = (df) donor matrix
        target = (df) target_matrix
        metrics = (list) list of metrics in donor/target matrix
        donor_ids = (list) donor ids
        target_ids = (list) target_ids
        T_0s = (list)
        singvals = (int) the number of singular values to keep; 0 if no HSVT
        """
        if (singvals != 0):
            self.donor = hsvt_new_df(donor, singvals)
        else:
            self.donor = donor
        self.target = target
        self.metrics = metrics
        self.donor_ids = donor_ids
        self.target_ids = target_ids
        self.num_k = len(self.metrics)
        self.T = int(self.target.shape[1]/self.num_k)
        self.T_0s = T_0s
        self.singvals = singvals
        
        """
        results are in a list of df's. df's.
        list[0] corresponds to the first T_0
        df's have target ids as index
        """
        self.pred = [pd.DataFrame(columns=self.target.columns, index=self.target.index)] * len(T_0s)
        self.betas = [pd.DataFrame(columns=self.donor.index, index=self.target.index)] * len(T_0s)
#         self.mse_all = [pd.DataFrame(columns=range(num_k), index=self.target.index)] * len(T_0s)
    
    def learn(self, target_id, T_0, method='lr'):
        # treatment unit
        y = self.target[self.target.index == target_id]
        y = y.values.flatten()

        # pre-intervention
        donor_pre = get_preint_data(self.donor.values, T_0, self.T, self.num_k)
        y_pre = get_preint_data(y, T_0, self.T, self.num_k)

        if (method == 'lr'):
            # linear regression
            regr = linear_model.LinearRegression(fit_intercept=False)
#             regr.n_jobs = -1
            regr.fit(donor_pre.T, y_pre)
            beta = regr.coef_
            
        elif (method == 'pinv'):
            beta = np.linalg.pinv(donor_pre.T).dot(y_pre)
            
        else:
            raise ValueError("Invalid method.")
        
        i = np.where(np.array(self.T_0s) == T_0)[0][0]
        
        # beta
        updated_beta = self.betas[i].copy()
        updated_beta[updated_beta.index == target_id] = [beta]
        self.betas[i] = updated_beta
        
        # prediction
        prediction = self.donor.T.dot(beta).values
        updated_pred = self.pred[i].copy()
        updated_pred[updated_pred.index == target_id] = [prediction]
        self.pred[i]= updated_pred
        
#         # mse
#         mse_list = []
#         for k in range(self.num_k):
#             val = mse(y[self.T*k:self.T*(k+1)], prediction[self.T*k:self.T*(k+1)])
#             mse_list.append(val)
#         updated_mse = self.mse_all[i].copy()
#         updated_mse[updated_mse.index == target_id] = [mse]
#         self.mse_all[i] = updated_mse

def getData(pre1, pre2, metrics, game_ids):
    """
        pre1 = (string) target or donor
        pre2 = (string) home or away
        metrics = (list) list of metrics
    """
    prefix = pre1+ "_" + pre2 + "_"
    df = pd.DataFrame()
    for i in range(len(metrics)):
        bucket = pd.read_pickle("../data/nba-hosoi/"+ prefix +metrics[i]+".pkl")
        df = pd.concat([df, bucket], axis = 1)
    df = df[df.index.isin(game_ids)]
    print("DataFrame size ", df.shape, "was created.")
    return df

# For Jehangir's Code
def getDataForGit(pre1, pre2, metrics, game_ids):
    """
        pre1 = (string) target or donor
        pre2 = (string) home or away
        metrics = (list) list of metrics
    """
    df_list = []
    prefix = pre1+ "_" + pre2 + "_"
    df = pd.DataFrame()
    for i in range(len(metrics)):
        df = pd.read_pickle("../data/nba-hosoi/"+ prefix +metrics[i]+".pkl")
        df = df.iloc[df.index.isin(game_ids)].T
        df = df.reset_index(drop=True)
        df_list.append(df)
    return df_list

def getDF(donor_list, target_list, target_id):
    # append the first column (target) before the donor dataframe
    num_k = len(donor_list)
    DF_list =[]
    for k in range(num_k):
        bucket = pd.concat([target_list[k][target_id], donor_list[k]], axis=1)
        DF_list.append(bucket)
    return DF_list

def DF_split(DF_list, T_0):
    # split train and test
    num_k = len(DF_list)
    DF_train_list =[]
    DF_test_list =[]
    for k in range(num_k):
        X = DF_list[k]
        train = X.iloc[:T_0,:]
        test = X.iloc[T_0:,:]
        DF_train_list.append(train)
        DF_test_list.append(test)
    return DF_train_list, DF_test_list


In [4]:
# experiment prarams
train_pcts = [0.1, 0.25, 0.5, 0.75, 0.9]
freq = 15
T = int(12*60*4/freq + 1)
T_0s = [int(np.ceil(train_pct * T)) for train_pct in train_pcts]
singvals = 1
donor_ids = np.array(pd.read_pickle('../data/nba-hosoi/donor_ids.pkl'))
target_ids = np.array(pd.read_pickle('../data/nba-hosoi/target_ids.pkl'))
metrics = ['points','assists', 'rebounds', 'bs', 'fouls']
num_k = len(metrics)
relative_weights = [1.0] * len(T_0s)

# data prep
donor_home_list = getDataForGit("donor", "home", metrics, donor_ids)
target_home_list = getDataForGit("target", "home", metrics, target_ids)

donor_away_list = getDataForGit("donor", "away", metrics, donor_ids)
target_away_list = getDataForGit("target", "away", metrics, target_ids)

In [5]:
donor_home_list

[nbaId  21300001  21300002  21300003  21300004  21300006  21300007  21300008  \
 0           0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 1           0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 2           2.0       2.0       2.0       0.0       0.0       0.0       2.0   
 3           2.0       2.0       2.0       0.0       0.0       0.0       2.0   
 4           2.0       2.0       2.0       0.0       0.0       0.0       2.0   
 5           5.0       2.0       2.0       0.0       0.0       0.0       2.0   
 6           5.0       2.0       2.0       0.0       0.0       0.0       2.0   
 7           5.0       2.0       2.0       2.0       0.0       3.0       4.0   
 8           7.0       2.0       4.0       2.0       0.0       3.0       4.0   
 9           8.0       2.0       4.0       2.0       0.0       3.0       4.0   
 10          8.0       2.0       4.0       2.0       0.0       3.0       4.0   
 11         10.0       2.0       4.0    